In [ ]:
from flickrapi import FlickrAPI
import pandas as pd
import tqdm, time, glob
from sqlalchemy import create_engine
import geopandas as gpd
from shapely.geometry import Point
import geopandas as gpd

FLICKR_PUBLIC = ''
FLICKR_SECRET = ''

flickr = FlickrAPI(FLICKR_PUBLIC, FLICKR_SECRET, format='parsed-json')
extras='url_sq,url_t,url_s,url_q,url_m,url_n,url_z,url_c,url_l,url_o'

In [ ]:
df = gpd.read_file('./shapefiles_lisbon/lisbon_city.shp')
df.to_crs(4326, inplace=True)
bb=list(df.bounds.iloc[0])
bb

In [ ]:
date_start = '2015-01-01'
date_end = '2022-03-20'
start_page = 10
end_page = 20
per_page = 100

df = pd.DataFrame(columns = ['flickr_id','title','desc','date','lat','lng','tags','locality','county','region','country','neighbourhood'])

In [ ]:
for word in ['alfama','mouraria']:
    for page_no in tqdm.tqdm(range(start_page, end_page)):

        flickr_dict = flickr.photos.search(min_upload_date=date_start, 
                                       max_upload_date=date_end,
                                       text=word, per_page=per_page, page=page_no,
                                       bbox=[bb[1], bb[0], bb[3], bb[2]], 
                                       accuracy=16, content_type=7, has_geo=1)
        time.sleep(10)
        for flicker_post in flickr_dict['photos']['photo']:
            photo = flickr.photos.getInfo(photo_id=flicker_post['id'])
            time.sleep(5)
            
            if 'location' in photo['photo'].keys():
                tag_content = []

                for tag in photo['photo']['tags']['tag']:
                    tag_content.append(tag['_content'])

                tag_content = ','.join(tag_content)

                df_row = [photo['photo']['id'],
                photo['photo']['title']['_content'],
                photo['photo']['description']['_content'],
                photo['photo']['dates']['taken'].split(' ')[0],
                photo['photo']['location']['latitude'],
                photo['photo']['location']['longitude'],
                tag_content]
                
                for location_key in ['locality','county','region','country','neighbourhood']:
                    if location_key in photo['photo']['location'].keys():
                        df_row.append(photo['photo']['location'][location_key]['_content'])
                    else:
                        df_row.append(None)
                        
                df.loc[len(df)] = df_row
            else:
                continue


In [ ]:
df.to_csv('./set_'+str(time.time()).split('.')[0]+'.csv', index=False)

In [ ]:
table_name = ""
database = ""
user = "postgres"
password = "postgres"
port = str(5432)
host = "localhost"

engine = create_engine("postgresql://"+user+":"+password+"@"+host+":"+port+"/"+database)

def generate_geometry(x):
    return Point(float(x['lat']), float(x['lng']))

In [ ]:
csv_files = glob.glob('set_*.csv')
failed = []

for fl_file in tqdm.tqdm(range(len(csv_files))):
    
    df = gpd.read_file('./set_1.csv')
    df['geometry'] = df.apply(generate_geometry, axis=1)
    df = df[df['geometry'].notna()]
    df.set_geometry('geometry', inplace=True)
    df.set_crs(epsg=4326, inplace=True, allow_override=True)
    gpd.GeoDataFrame.to_postgis(df, name = table_name, con = engine, schema = 'public', if_exists = 'append')
